## 1. Defining algebraic governing equations

### Old library

In [1]:
from modeling.compute import Var
from modeling.gen1.compute import eqvar
from trash.inputresolver import eqvars, default_out, default_in

In [2]:
x,a,c = Var('x', 2), Var('a'), Var('c')
y, eq0 = eqvar('y', x**2)
b, eq1 = eqvar('b', 2*a-x)
eq2 = (c, b**2)
eq3 = (a, c*b)
eqs ={idx:eq for idx,eq in enumerate([eq0,eq1,eq2,eq3])}

In [3]:
eqvars(eqs), default_in(eqs), default_out(eqs)

({0: {x, y}, 1: {a, b, x}, 2: {b, c}, 3: {a, b, c}},
 {x},
 {0: y, 1: b, 2: c, 3: a})

### New library

In [4]:
from engine.torchengine import AnalyticalSetSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, ElimResidual
from engine.torchdata import symbols, reverse_indices, load_vals
from graph.operators import invert_edges
from graph.graphutils import sources

In [5]:
x, a, c, y, b, indices = symbols('x, a, c, y, b')
eq0 = AnalyticalSetSympy(x**2, y, indices)
eq1 = AnalyticalSetSympy(2*a-x, b, indices)
eq2 = AnalyticalSetSympy(b**2, c, indices)
eq3 = AnalyticalSetSympy(c*b, a, indices)
sets = {idx:eq for idx,eq in enumerate([eq0,eq1,eq2,eq3])}

In [6]:
edges_in = {idx: reverse_indices(s.analysis.structure[0], indices) for idx,s in sets.items()}
edges_out = {idx: reverse_indices(s.analysis.structure[1], indices) for idx,s in sets.items()}
edges = (edges_in, edges_out)

In [7]:
invert_edges(*edges)[0], sources(*edges), edges_out

({0: (x, y), 1: (a, x, b), 2: (b, c), 3: (b, c, a)},
 {x},
 {0: (y,), 1: (b,), 2: (c,), 3: (a,)})

## 2. Multidisciplinary analysis (without any elimination)

### New library

In [8]:
import torch
import numpy as np

In [9]:
R = EliminateAnalysisMergeResiduals(functions=[s.residual for s in sets.values()])
solvefor = torch.tensor([s.analysis.structure[1] for s in sets.values()])
S = ElimResidual(R, solvefor, indices, solvefor_raw=True)
x0= load_vals({'x': 2}, indices, default=1, isdict=True)
S(x0)

tensor([2.0000, 1.5827, 1.3581, 4.0000, 1.1654], dtype=torch.float64)

### Old library (with openMDAO)

In [1]:
import openmdao.api as om
from problemgen.gen1.openmdao import buildidpvars, coupled_run
from problemgen.gen1.resultscheck import get_outputs, check_eqs

In [5]:
prob = om.Problem()
model = prob.model
dins = default_in(eqs)
buildidpvars(dins, model)
counter = coupled_run(eqs, [], solve_order=(0,1,2,3), parent=model, 
                      root=model, maxiter=20)
prob.setup();

In [8]:
prob.run_model()


group1
NL: Newton Converged in 7 iterations


In [9]:
prob.model.list_outputs();

5 Explicit Output(s) in 'model'

varname  val         
-------  ------------
inp
  x      [2.]        
group1
  eq0
    y    [4.]        
  eq1
    b    [1.16537304]
  eq2
    c    [1.35809433]
  eq3
    a    [1.58268652]


0 Implicit Output(s) in 'model'




In [10]:
check_eqs(eqs, get_outputs(eqs, model, varasstring=True))

{0: (4.0, 4.0),
 1: (1.1653730430698817, 1.1653730431232594),
 2: (1.3580943295139563, 1.3580943295139563),
 3: (1.5826865215616297, 1.5826865215616297)}

## 3. MDA with elimination (tearing)

### Old library

In [31]:
from modeling.compute import Evaluable
from functools import reduce
import numpy as np
import scipy

In [13]:
r = [left-right for left, right in eqs.values()]
ev = [Evaluable.fromsympy(elt) for elt in r] #residuals
ex = [Evaluable.fromsympy(right) for left, right in eqs.values()]

In [34]:
def resolve_eliminated_implicit(eqs, elim_order, impl_eqs):
    inp = default_in(eqs)
    sols = {elt.name: elt.varval for elt in inp}
    expected_ex = default_in([eqs[elt] for elt in elim_order], eqdictin=False)
    ex_out = {eqs[elt][0] for elt in elim_order}
    expected_im = reduce(lambda a,b: a.union(b), [eqs[elt][1].free_symbols for elt in impl_eqs])
    inp_impl = expected_ex.union(expected_im) -ex_out - inp
    inp_impl_idx = list(enumerate(inp_impl))
    r = [left-right for left, right in eqs.values()]
    ev = [Evaluable.fromsympy(elt) for elt in r]
    def only_a(a):
        sols.update({elt.name: a[idx] for idx,elt in inp_impl_idx})
        for elt in elim_order:
            sols.update({eqs[elt][0].name : ex[elt].evaldict(sols)})
        return  np.array([ev[evi].evaldict(sols) for evi in impl_eqs])
    return only_a

In [35]:
only_a = resolve_eliminated_implicit(eqs, [0,1], [2,3]) # eliminates eq 0
scipy.optimize.fsolve(only_a, np.array([1., 1.]))

array([1.35809433, 1.58268652])

### New library

In [51]:
import torch

In [52]:
R = EliminateAnalysisMergeResiduals(
    analyses=[sets[idx].analysis for idx in [0,1]],
    functions=[sets[idx].residual for idx in [2,3]])
solvefor = torch.tensor([sets[idx].analysis.structure[1] for idx in [2,3]])
S = ElimResidual(R, solvefor, indices, solvefor_raw=True)
x0= load_vals({'x': 2}, indices, default=1, isdict=True)
S(x0)

tensor([2.0000, 1.5827, 1.3581, 1.0000, 1.0000], dtype=torch.float64)

**Option to update elimination vars**

In [55]:
A = EliminateAnalysis([sets[idx].analysis for idx in [0,1]]+[S])
A(x0)

tensor([2.0000, 1.5827, 1.3581, 4.0000, 1.1654], dtype=torch.float64)